In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import cv2
import time
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras.models import load_model
import tensorflow as tf
import os

In [ ]:
# CNN Classes
signs = pd.read_csv('yolo-coco-data/label_names.csv')

# YOLO Classes
with open('yolo-coco-data/coco.names') as f:
    labels = [line.strip() for line in f]

# CNN Model
model = load_model('yolo-coco-data/model.h5')

# Pathes to CFG file and Weights
path_to_weights = 'yolo-coco-data/yolo.weights'
path_to_cfg = 'yolo-coco-data/yolo.cfg'

# Load YOLO
network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)

In [ ]:
# To use with GPU
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)

# Getting names of all YOLO layers
layers_all = network.getLayerNames()

# Getting Unconnected Layers YOLO
layers_names_output = [layers_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

In [ ]:
# Minimum Probability to eliminate weak predictions
probability_minimum = 0.2

# Threshold for filtering weak bounding boxes with maximum supression
threshold = 0.2

# Random Colors for each yolo Class
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

In [ ]:
# Real Time Video Stream
camera = cv2.VideoCapture(0)

# Variables for frame dimensions
h, w = None, None

# Writer for saving video
writer = None

# Setting default size of plots
plt.rcParams['figure.figsize'] = (3, 3)

# Frame Count
f = 0

# Time spent
t = 0

# Setting default size of plots
plt.rcParams['figure.figsize'] = (3, 3)

In [ ]:
# Function for Grayscaling the image
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

# Function for equalizing image using Histogram Equalizer
def equalize(img):
    img = cv2.equalizeHist(img)
    return img

# Preprocessing function that uses Grayscaling, Equalizing and Normalizing by division to 255
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

In [ ]:
# Catching frames in the loop
while True:
    # Capturing frames one-by-one
    ret, frame = camera.read()

    # If the frame was not retrieved
    if not ret:
        break

    # Getting spatial dimensions of the frame for the first time
    if w is None or h is None:
        # Slicing two elements from tuple
        h, w = frame.shape[:2]

    # Blob from current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # Forward pass with blob through output layers
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increasing counters
    f += 1
    t += end - start

    # Spent time for current frame
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))

    # Lists for detected bounding boxes, confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 80 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]

            # Eliminating weak predictions by minimum probability
            if confidence_current > probability_minimum:
                # Scaling bounding box coordinates to the initial frame size
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Getting top left corner coordinates
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)

    # Implementing non-maximum suppression of given bounding boxes
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    # Checking if there is any detected object been left
    if len(results) > 0:
        # Going through indexes of results
        for i in results.flatten():
            # Bounding box coordinates, its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            # If traffic sign is detected
            if(class_numbers[i] == 8):
                # Cut fragment with Traffic Sign
                c_ts = frame[y_min:y_min + int(box_height), x_min:x_min + int(box_width), :]

                if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                    pass
                else:
                    # Preprocessing Frame
                    img = np.asarray(c_ts)
                    img = cv2.resize(img, (32, 32))
                    img = preprocessing(img)
                    img = img.reshape(1, 32, 32, 1)

                    # Feeding to the Keras CNN model to get predicted label among 81 classes
                    scores = model.predict(img)

                    # Scores is given for image with 81 numbers of predictions for each class
                    # Getting only one class with maximum value
                    prediction = np.argmax(scores)

                    # Colour for current bounding box
                    colour_box_current = colours[class_numbers[i]].tolist()

                    # Drawing bounding box on the original current frame
                    cv2.rectangle(frame, (x_min, y_min),
                                  (x_min + box_width, y_min + box_height),
                                  colour_box_current, 2)

                    # Preparing text with label and confidence for current bounding box
                    text_box_current = '{}: {:.4f}'.format(signs['SignName'][prediction],
                                                           confidences[i])

                    # Putting text with label and confidence on the original image
                    cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)
            else:
                # Preparing colour for current bounding box
                # and converting from numpy array to list
                colour_box_current = colours[class_numbers[i]].tolist()

                # Drawing bounding box on the original current frame
                cv2.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              colour_box_current, 2)

                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                                       confidences[i])

                # Putting text with label and confidence on the original image
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)

    # Showing results obtained from camera in Real Time
    cv2.namedWindow('YOLO Real Time Detections', cv2.WINDOW_NORMAL)
    cv2.imshow('YOLO Real Time Detections', frame)

    # Breaking the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# Releasing video reader and writer
camera.release()
cv2.destroyAllWindows()